In [4]:
import os
from typing import Type
import requests
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo-0125"
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        # ddg = DuckDuckGoSearchAPIWrapper()
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company. Example: AAPL, TSLA", )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()
        

class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]
        
        
class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformanceTool"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()
        


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[StockMarketSymbolSearchTool(),
           CompanyOverviewTool(),
           CompanyIncomeStatementTool(),
           CompanyStockPerformanceTool(),],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`


[snippet: The latest Cloudflare stock prices, stock quotes, news, and NET history to help you invest and trade smarter. ... Cloudflare's market capitalization is $29.75 B by 298.38 M shares outstanding. Is ..., title: Cloudflare Stock Price | NET Stock Quote, News, and History | Markets ..., link: https://markets.businessinsider.com/stocks/net-stock?op=1], [snippet: Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile., title: NET: Cloudflare Inc Stock Price Quote - New York - Bloomberg, link: https://www.bloomberg.com/quote/NET:US], [snippet: Internet firm Cloudflare said in a statement on Thursday that an advanced group of hackers tried to burrow deep into its global network late last year but were thwarted. 2 months ago - Reuters. Get a real-time Cloudfl

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc. Financial Information:\n\n- **Company Overview:**\n  - **Name:** Cloudflare Inc\n  - **Symbol:** NET\n  - **Sector:** Technology\n  - **Industry:** Services-Prepackaged Software\n  - **Market Capitalization:** $28.4 billion\n  - **Revenue (TTM):** $1.3 billion\n  - **Gross Profit (TTM):** $989.7 million\n  - **EPS (TTM):** -$0.55\n  - **Analyst Target Price:** $100.19\n  - **52-Week High:** $116\n  - **52-Week Low:** $39.9\n\n- **Income Statement:**\n  - **Gross Profit (2023):** $989.7 million\n  - **Total Revenue (2023):** $1.3 billion\n  - **Operating Income (2023):** -$185.5 million\n  - **Net Income (2023):** -$183.9 million\n\n- **Stock Performance:**\n  - **50-Day Moving Average:** $96.44\n  - **200-Day Moving Average:** $76.12\n  - **Current Price:** [Check the la